In [24]:
#Create dimensions
from TM1py.Services import TM1Service
from TM1py.Objects import Cube, Dimension, Hierarchy, Element, ElementAttribute, Subset
from TM1py.Utils import Utils

#Input authentication based on your server
with TM1Service(address='localhost', port=8801, user='admin', password='', ssl=True) as tm1:
    Dimension_1 = 'Year'
    Dimension_2 = 'Month'
 
    def year_creation():
        dim_name = Dimension_1
        #Define an element name
        new_dim = dim_name
        
        #Define elements for the dimension
        elements = [Element('{}'.format(i), 'Numeric') for i in range(2019, 2025)]
        
        #Define parent
        parent = 'Total'
        
        #Append parent into the element list
        elements.append(Element(parent, 'Consolidated'))
        
        #Create rollup dictionary
        edges = Utils.CaseAndSpaceInsensitiveTuplesDict()
        
        #Create a loop to fill rollup dictionary
        for i in range(2019,2025):
            parent_child = (parent, '{}'.format(i))
            edges[parent_child] = 1
            
        #Create dimension attributes
        element_attributes = [ElementAttribute(name='Previous Year', attribute_type='String'),
                      ElementAttribute(name='Next Year', attribute_type='String')]
        
        #Define hierarchy using object above
        hierarchy = Hierarchy(name=new_dim, dimension_name=new_dim, elements=elements, edges=edges, element_attributes=element_attributes)
        
        #Define dimension using dimension name and created hierarchy
        d = Dimension(name=new_dim,hierarchies=[hierarchy])
        
        #Check if the dimension exists, delete and create a new one. If not, create a new one
        if(tm1.dimensions.exists(new_dim)==False):
            tm1.dimensions.create(d)
        else:
            tm1.dimensions.delete(new_dim)
            tm1.dimensions.create(d)
            
        #Fill previous year and next year attributes
        cellset={}
        for i in range(2019,2025):
            cube = ("}}ElementAttributes_{}").format(new_dim)
            cellset[(i,'Previous Year')] = i-1
            cellset[(i,'Next Year')] = i+1
            tm1.cubes.cells.write_values(cube, cellset)

    def month_creation():
        dim_name = Dimension_2
        #Define dimension name
        new_dim = dim_name
        
        #Define month list 
        period = ['01','02','03','04','05','06','07','08','09','10','11','12']
        short_name = ['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec']
        full_name = ['January','February','March','April','May','June','July','August','September','October','November','December']
        
        #Define element for each item in period list
        elements = [Element('{}'.format(i), 'Numeric') for i in period]
        
        #Define parents
        parent = 'Total Year'
        Q1 = 'Q1'
        Q2 = 'Q2'
        Q3 = 'Q3'
        Q4 = 'Q4'
        
        #Add parents into the element list
        elements.append(Element(parent, 'Consolidated'))
        elements.append(Element(Q1, 'Consolidated'))
        elements.append(Element(Q2, 'Consolidated'))
        elements.append(Element(Q3, 'Consolidated'))
        elements.append(Element(Q4, 'Consolidated'))
        
        #Create consolidation dictionary
        edges = Utils.CaseAndSpaceInsensitiveTuplesDict()
        
        #Create total 1 year consolidation for all elements
        for i in period:
            parent_child = (parent, '{}'.format(i))
            edges[parent_child] = 1
        #Create Quarter 1 consolidation
        for i in period[:3]:
            parent_child = (Q1, '{}'.format(i))
            edges[parent_child] = 1
        #Create Quarter 2 consolidation
        for i in period[3:6]:
            parent_child = (Q2, '{}'.format(i))
            edges[parent_child] = 1            
        #Create Quarter 3 consolidation
        for i in period[6:9]:
            parent_child = (Q3, '{}'.format(i))
            edges[parent_child] = 1    
        #Create Quarter 4 consolidation
        for i in period[9:12]:
            parent_child = (Q4, '{}'.format(i))
            edges[parent_child] = 1
        #Create Year to Date consolidation
        for i in period:
            YTD = '{} YTD'.format(i)
            elements.append(Element(YTD, 'Consolidated'))
            temp = period.index(i)+1
            res = period[:temp]
            for j in res:
                parent_child = (YTD, '{}'.format(j))
                edges[parent_child] = 1
                
        #Attribute creation
        element_attributes = [ElementAttribute(name='Previous Month', attribute_type='String'),
                      ElementAttribute(name='Next Month', attribute_type='String'),
                      ElementAttribute(name='Short Name', attribute_type='Alias'),
                      ElementAttribute(name='Full Name', attribute_type='Alias')]
        
        #Define hierarchy using object above
        hierarchy = Hierarchy(name=new_dim, dimension_name=new_dim, elements=elements, edges=edges, element_attributes=element_attributes)
        
        #Define dimension using dimension name and hierarchy
        d = Dimension(name=new_dim,hierarchies=[hierarchy])
        
        #Check if the dimension exists, delete and create a new one. If not, create a new one
        if(tm1.dimensions.exists(new_dim)==False):
            tm1.dimensions.create(d)
        else:
            tm1.dimensions.delete(new_dim)
            tm1.dimensions.create(d)    
        
        #Fill alias, previous month and next month attributes
        cellset={}
        for i in period:
            cube = ("}}ElementAttributes_{}").format(new_dim)
            index = period.index(i)
            if(i != '01'):
                prev_mth = period[index-1]
                cellset[(i,'Previous Month')] = prev_mth
            if(i != '12'):
                next_mth = period[index+1]
                cellset[(i,'Next Month')] = next_mth
            fn = full_name[index]
            sn = short_name[index]
            cellset[(i,'Short Name')] = sn
            cellset[(i,'Full Name')] = fn
            
            tm1.cubes.cells.write_values(cube, cellset)
        
        #Create subset for leaf element
        s = Subset(dimension_name=new_dim, subset_name='Leaf Element', alias='Short Name', elements=period)
        tm1.dimensions.subsets.create(subset=s, private=False)      
          
    #Dimension validation
    if(tm1.dimensions.exists(Dimension_1)==False):
        year_creation()
        print('Dimension {} has been created'.format(Dimension_1))
    else:
        print('Dimension {} already existed'.format(Dimension_1))
        print('Do you want to rebuild dimension?')
        choice = input('Yes/No')
        if(choice == 'Yes'.lower()):
            tm1.dimensions.delete(Dimension_1)
            year_creation()
            print('Dimension {} has been created\n'.format(Dimension_1))
               
    if(tm1.dimensions.exists(Dimension_2)==False):
        month_creation()
        print('Dimension {} has been created'.format(Dimension_2))
    else:
        print('Dimension {} already existed'.format(Dimension_2))
        print('Do you want to rebuild dimension?')
        choice = input('Yes/No')
        if(choice == 'Yes'.lower()):
            tm1.dimensions.delete(Dimension_2)
            month_creation()
            print('Dimension {} has been created\n'.format(Dimension_2))


Dimension Year already existed
Do you want to rebuild dimension?
Yes/Noyes
Dimension Year has been created

Dimension Month already existed
Do you want to rebuild dimension?
Yes/Noyes
Dimension Month has been created

